# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [67]:
import pandas as pd
import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
from ipywidgets import interact
import plotly.graph_objects as go

cf.go_offline()

In [80]:
data = pd.read_excel('../data/Online Retail.xlsx')
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [73]:
data_1 = data[(data['Country'] != 'United Kingdom')]
data_1 = data_1[(data_1['InvoiceDate'] >= '2011-04-01') & (data_1['InvoiceDate'] <= '2011-04-30')]
data_1 = data_1.groupby('Country').agg({'Revenue':'sum', 'Quantity':'sum'})
data_1

,Revenue,Quantity
Country,,
Australia,421.60,224
Austria,584.78,308
Belgium,1788.48,1170
Brazil,1143.60,356
Channel Islands,243.00,96
EIRE,7270.50,4129
Finland,1368.92,810
France,3899.31,2265
Germany,10994.79,5702


In [74]:
fig = px.bar(data_1, x=data_1.index, y='Revenue')
fig.show()

In [75]:
fig = go.Figure(data=[go.Bar(name='Revenue', x=data_1.index, y=data_1.Revenue), 
                    go.Bar(name='Quantity', x=data_1.index, y=data_1.Quantity)])

fig.show()

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [78]:
data_2 = data[(data['Country'] == 'France')]
data_2 = data_2[(data_2['InvoiceDate'] >= '2011-01-01') & (data_2['InvoiceDate'] <= '2011-05-31')]
data_2 = data_2.groupby('InvoiceDate').agg({'Revenue':'sum', 'Quantity':'sum'})

In [79]:
fig = go.Figure(data=[go.Line(name='Revenue', x=data_2.index, y=data_2.Revenue), 
                    go.Line(name='Quantity', x=data_2.index, y=data_2.Quantity)])

fig.show()

/usr/local/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [86]:
date_3 = data[(data['Description'] == 'PARTY BUNTING')]
date_3 = date_3.groupby('Country').mean()
date_3.head()

,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
Country,,,,,
Australia,554329.625000,33.125000,4.7125,143.78125,12399.250000
Austria,552202.000000,8.000000,4.9500,39.60000,12414.000000
Belgium,557600.000000,4.000000,4.9500,19.80000,12363.000000
Channel Islands,561821.666667,13.333333,4.9500,66.00000,14934.000000
Cyprus,553141.333333,2.333333,4.7500,10.95000,12373.333333


In [95]:
fig = px.scatter(date_3, x='Quantity', y='UnitPrice', color=date_3.index, size='Quantity', hover_data=['Revenue'])
fig.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [130]:

date_41 = data[(data['Country'] == 'EIRE')]
date_42 = data[(data['Country'] == 'Germany')]
date_43 = data[(data['Country'] == 'France')]
date_44 = data[(data['Country'] == 'Netherlands')]
date_41.head()
date_42.head()
date_43.head()
date_44.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands
534,546027,2011-03-09 08:09:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,12759,Netherlands
606,547365,2011-03-22 12:36:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,12,2.95,35.4,12775,Netherlands
1111,557525,2011-06-21 09:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,12759,Netherlands
1672,571697,2011-10-18 14:49:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,12759,Netherlands


In [139]:
date_4 = data[data['Country'].isin(['EIRE', 'Germany', 'France', 'Netherlands'])]
quantityperinvoice = fewcountries.pivot_table(values='Quantity', columns='Country', 
                      index='InvoiceNo', aggfunc='sum')
quantityperinvoice.iplot(kind='hist', xTitle='Quantity of Invoice', 
           yTitle='Amount of invoices', title='Distribution of invoice quantities')		

NameError: name 'fewcountries' is not defined

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.